In [120]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

metadata 읽기

In [121]:
meta = pd.read_csv( '/kaggle/input/the-movies-dataset/movies_metadata.csv' )
meta

metadata에서 필요한 열(column)만 추려내기

In [122]:
meta = meta[ ['id', 'original_title', 'original_language', 'genres'] ]
meta

열(column) 이름 변경하기

In [123]:
meta = meta.rename(columns={'id':'movieId',
 'original_title': 'title',
'original_language': 'language'})
meta

language가 'en'인 데이터만 솎아내기

In [124]:
meta = meta.loc[meta['language'] == 'en',:]
meta

movieId의 타입을 숫자로 변경하기

In [125]:
meta.movieId = pd.to_numeric(meta.movieId)
meta.movieId

genre 정제하기: json string을 python set으로 변환하는 함수 구현

In [126]:
def str_to_set(x):
    genre_set = set()
    for item in eval(x):
        genre_set.add(item['name'])
    return genre_set

genre 정제하기: meta.genres의 모든 값에 str_to_set 적용

In [127]:
meta.genres = meta.genres.apply(str_to_set)
meta

keywords 데이터 읽기

In [128]:
keywords = pd.read_csv( '/kaggle/input/the-movies-dataset/keywords.csv' )
keywords

keywords 데이터의 keywords를 집합으로 변환하기

In [129]:
keywords.keywords = keywords.keywords.apply(str_to_set)
keywords

id 열을 movieId로 이름변경하고 데이터타입을 숫자로 변환하기

In [130]:
keywords = keywords.rename(columns={'id': 'movieId'})
keywords.movieId = pd.to_numeric(keywords.movieId)
keywords

같은 movieId를 갖는 데이터를 합치기

In [131]:
meta = pd.merge(meta, keywords, on='movieId', how='inner')
meta

영화 찾아보고 합쳐서 출력해보기

In [132]:
dk = meta.loc[meta.title == 'The Dark Knight'].iloc[0]
dkr = meta.loc[meta.title == 'The Dark Knight Rises'].iloc[0]
pd.concat([dk, dkr], axis=1).T

Jaccard 유사도 함수 구현 및 실행

In [133]:
def jaccard_similarity(s1, s2):
    if len(s1|s2) == 0:
        return 0
    return len(s1&s2)/len(s1|s2)

jaccard_similarity(dk.genres|dk.keywords,
dkr.genres|dkr.keywords)

Rating 데이터 읽기

In [134]:
ratings = pd.read_csv( '/kaggle/input/the-movies-dataset/ratings_small.csv' )
ratings

Rating의 movieId를 숫자로 변환

In [135]:
ratings.movieId = pd.to_numeric(ratings.movieId)

metadata로부터 title 정보 가져와 합치기

In [136]:
ratings = pd.merge(ratings, meta[['movieId', 'title']], on='movieId', how='inner')

pivot table 만들기

In [137]:
matrix = ratings.pivot_table(index= 'userId', columns='title',
values='rating')
matrix

Pearson 유사도 함수 구현

In [138]:
def pearson_similarity(u1, u2):
    u1_c = u1 - u1.mean()
    u2_c = u2 - u2.mean()
    denom = np.sqrt(np.sum(u1_c ** 2) * np.sum(u2_c ** 2))
    if denom != 0:
        return np.sum(u1_c * u2_c)/denom
    else:
        return 0

The Dark Knight와 Prom Night의 Pearson 유사도 구해보기

In [139]:
dk_rating = matrix['The Dark Knight']
pk_rating = matrix['Prom Night']
pearson_similarity(dk_rating, pk_rating)

비슷한 영화 추천 기능 구현

In [140]:
def find_similar_movies (input_title , matrix, n, alpha):
    input_meta = meta.loc[ meta[ 'title'] == input_title].iloc[ 0]
    input_set = input_meta.genres | input_meta.keywords

    result = []

    for this_title in matrix.columns:
        if this_title == input_title:
            continue
            
        this_meta = meta.loc[ meta[ 'title'] == this_title].iloc[ 0]
        this_set = this_meta.genres | this_meta.keywords

        pearson = pearson_similarity(matrix[this_title], matrix[input_title])
        jaccard = jaccard_similarity(this_set, input_set)

        score = alpha * pearson + ( 1-alpha) * jaccard
        result.append( (this_title, pearson, jaccard, score) )

    result.sort(key= lambda r: r[3], reverse= True)

    return result[:n]

* input_meta: 입력된 영화의 metadata
* input_set: 입력된 영화의 genres와 keyword의 합집합

* result: 모든 영화마다 유사도를 계산하여 저장
* 입력된 영화는 유사도 계산을 하지 않고 Pass~

* this_meta: 유사도를 계산하려는 영화의 metadata
* this_set: 이 영화의 genres와 keywords의 합집합

* pearson: 입력 영화와 이번 영화의 pearson 유사도 결과
* jaccard: 입력 영화와 이번 영화의 jaccard 유사도 결과

* score: pearson 점수와 jaccard 점수의 가중치 합
* result에 계산 결과 추가

* 모든 영화에 대해 유사도 계산이 끝나면 result를 정령
* 상위 n개를 return

The Dark Knight와 비슷한 영화 추천해보기

In [141]:
result = find_similar_movies('The Dark Knight', matrix, 10, 0.3)
pd.DataFrame(result, columns = ['title', 'pearson', 'jaccard', 'score'])